In [4]:
import pandas as pd
logs = pd.read_csv('/Users/sergejvasiljev/Documents/DATASETS/ml-latest-small/visit_log.csv', sep = ';')

### Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:
  
    если источник traffic_source равен yandex или google, то в source_type ставится organic  
    для источников paid и email из России - ставим ad  
    для источников paid и email не из России - ставим other  
    все остальные варианты берем из traffic_source без изменений


In [5]:
logs.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [21]:
logs.source_type = logs.traffic_source
logs.loc[logs.traffic_source == ('yandex' or 'google'),'source_type'] = 'organic'
logs.loc[((logs.traffic_source == ('paid' or 'email')) & (logs.region == 'Russia')), 'source_type'] = 'ad'
logs.loc[((logs.traffic_source == ('paid' or 'email')) & (logs.region != 'Russia')), 'source_type'] = 'other'
logs

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
...,...,...,...,...,...,...,...
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,other
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,ad
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex,organic
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,ad


### Задание 2
  
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:
  
- Прочитайте содержимое файла с датафрейм  
- Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном  

In [25]:
URLs = pd.read_csv('/Users/sergejvasiljev/Documents/DATASETS/ml-latest-small/URLs.txt')

In [23]:
URLs.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [35]:
URLs.url.str.contains('/\d{8}-')

0     False
1     False
2     False
3      True
4      True
      ...  
88     True
89     True
90     True
91     True
92     True
Name: url, Length: 93, dtype: bool

In [36]:
URLs.url.str.contains('/\d{8}-').value_counts()

True     84
False     9
Name: url, dtype: int64

### Задание 3
  
- Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [37]:
ratings = pd.read_csv('/Users/sergejvasiljev/Documents/DATASETS/ml-latest-small/ratings.csv')

In [48]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [86]:
df_max = ratings[['userId', 'timestamp']].groupby('userId').max().reset_index()
df_min = ratings[['userId', 'timestamp']].groupby('userId').min().reset_index()

In [96]:
df_index = ratings.groupby('userId').timestamp.count().reset_index()
df_total = df_index.loc[df_index.timestamp > 100].join(df_max, how='left', rsuffix='_max').join(df_min, how='left',rsuffix='_min')

In [100]:
df_total['live_time'] = df_total.timestamp_max - df_total.timestamp_min
df_total.head()

,userId,timestamp,userId_max,timestamp_max,userId_min,timestamp_min,live_time
3,4,204,4,949982274,4,949778714,203560
7,8,116,8,1154474527,8,1154389340,85187
14,15,1700,15,1469330735,15,997937239,471393496
16,17,363,17,1127476640,17,1127468587,8053
18,19,423,19,855195373,19,855190091,5282


In [99]:
df_total.live_time.mean()

40080507.4496124

### Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:  
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента  
_Обратите внимание, что в процессе объединения таблиц данные не должны теряться._

In [107]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

In [108]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

In [109]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

In [110]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [115]:
rzd.merge(auto, how='outer', on='client_id').merge(air, how='outer', on='client_id').fillna(0)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [117]:
rzd.merge(auto, how='outer', on='client_id').merge(air, how='outer', on='client_id').merge(client_base, how='left', on='client_id').fillna(0)

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4
